#**전제조건**<Br>
사람 파일은 human1.jpg<br>
배경 파일은 back1.jpg<br> 
런타임은 gpu
<br>
<br>
#**함수 설명**<br>


*   style_transfer: anime gan 실행
*   segmentation(file_name): segmentation 실행
*   pre_processing(black_mask, ani_img): 이미지 전처리(흰 배경으로 만들기) 실행


# 라이브러리

In [1]:
import os
import gdown
import shutil 
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import subprocess

In [3]:
!pip install flask_ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

In [4]:
os.mkdir('templates')
os.mkdir('static')
%cd static/
os.mkdir('anime_outputs')
os.mkdir('seg_outputs')
os.mkdir('splicing_outputs')
os.mkdir('inputs')
%cd /content/

/content/static
/content


# Github 불러오기

In [2]:
!pip install dominate==2.4.0
!python -m pip install visdom


     |████████████████████████████████| 686kB 15.5MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp37-none-any.whl size=655251 sha256=f8ec2010eec32e4c98eb51d5bfe02e059cdfc2b43b45df334a59613a2a69af22
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5712 sha256=3c68bcc9033f82003b559538bc02616f9f933a3d6bb5079b05b126358850d529
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile


In [5]:
!git clone https://github.com/TachibanaYoshino/AnimeGANv2

Cloning into 'AnimeGANv2'...
remote: Enumerating objects: 1679, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1679 (delta 0), reused 0 (delta 0), pack-reused 1676
Receiving objects: 100% (1679/1679), 996.82 MiB | 31.79 MiB/s, done.
Resolving deltas: 100% (110/110), done.
Checking out files: 100% (326/326), done.


In [6]:
!pip install tensorflow==1.15
!git clone https://github.com/bryandlee/animegan2-pytorch.git
%cd animegan2-pytorch
!mkdir inputs
!mkdir outputs
%cd ..

     |████████████████████████████████| 412.3MB 33kB/s 
     |████████████████████████████████| 3.8MB 40.7MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 512kB 45.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=1a59ef08cec2cb9d3aaffd5833e6859a745b045e573ea501c14ac81da12e86bf
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensor

In [7]:
!pip install ninja
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing
!mkdir checkpoints
!mkdir inputs
!mkdir outputs
%cd ..

     |████████████████████████████████| 112kB 14.0MB/s 
Cloning into 'Self-Correction-Human-Parsing'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 719 (delta 3), reused 5 (delta 0), pack-reused 708
Receiving objects: 100% (719/719), 3.80 MiB | 14.23 MiB/s, done.
Resolving deltas: 100% (145/145), done.
/content/Self-Correction-Human-Parsing
/content


# Style Transfer

In [8]:
%cd /content
%cd animegan2-pytorch

/content
/content/animegan2-pytorch


In [9]:
text = '''
#!/usr/bin/env python
# -*- encoding: utf-8 -*-

import sys
sys.path.append('/content')

import argparse
import numpy as np
import os

import tensorflow as tf
from AnimeGANv2.net import generator as tf_generator

import torch
from model import Generator


def load_tf_weights(tf_path):
    test_real = tf.placeholder(tf.float32, [1, None, None, 3], name='test')
    with tf.variable_scope("generator", reuse=False):
        test_generated = tf_generator.G_net(test_real).fake

    saver = tf.train.Saver()

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, device_count = {'GPU': 0})) as sess:
        ckpt = tf.train.get_checkpoint_state(tf_path)

        assert ckpt is not None and ckpt.model_checkpoint_path is not None, f"Failed to load checkpoint {tf_path}"

        saver.restore(sess, ckpt.model_checkpoint_path)
        print(f"Tensorflow model checkpoint {ckpt.model_checkpoint_path} loaded")

        tf_weights = {}
        for v in tf.trainable_variables():
            tf_weights[v.name] = v.eval()
    
    return tf_weights

            
def convert_keys(k):

    # 1. divide tf weight name in three parts [block_idx, layer_idx, weight/bias]
    # 2. handle each part & merge into a pytorch model keys
    
    k = k.replace("Conv/", "Conv_0/").replace("LayerNorm/", "LayerNorm_0/")    
    keys = k.split("/")[2:]
    
    is_dconv = False

    # handle C block..
    if keys[0] == "C":
        if keys[1] in ["Conv_1", "LayerNorm_1"]:
            keys[1] = keys[1].replace("1", "5")
        
        if len(keys) == 4:
            assert "r" in keys[1]

            if keys[1] == keys[2]:
                is_dconv = True
                keys[2] = "1.1"
            
            block_c_maps = {
                "1":  "1.2",
                "Conv_1":  "2",
                "2":  "3",
            }
            if keys[2] in block_c_maps:
                keys[2] = block_c_maps[keys[2]]

            keys[1] = keys[1].replace("r", "") + ".layers." + keys[2]
            keys[2] = keys[3]
            keys.pop(-1)
    assert len(keys) == 3

    # handle output block
    if "out" in keys[0]:
        keys[1] = "0"
    
    # first part
    if keys[0] in ["A", "B", "C", "D", "E"]:
        keys[0] = "block_" + keys[0].lower()        
        
    # second part
    if "LayerNorm_" in keys[1]:
        keys[1] = keys[1].replace("LayerNorm_", "") + ".2"
    if "Conv_" in keys[1]:
        keys[1] = keys[1].replace("Conv_", "") + ".1"
        
    # third part
    keys[2] = {
        "weights:0": "weight",
        "w:0": "weight",
        "bias:0": "bias",
        "gamma:0": "weight",
        "beta:0": "bias",
    }[keys[2]]
        
    return ".".join(keys), is_dconv


def convert_and_save(tf_checkpoint_path, save_name):

    tf_weights = load_tf_weights(tf_checkpoint_path)
    
    torch_net = Generator()
    torch_weights = torch_net.state_dict()

    torch_converted_weights = {}
    for k, v in tf_weights.items():
        torch_k, is_dconv = convert_keys(k)
        assert torch_k in torch_weights, f"weight name mismatch: {k}"

        converted_weight = torch.from_numpy(v)
        if len(converted_weight.shape) == 4:
            if is_dconv:
                converted_weight = converted_weight.permute(2, 3, 0, 1)
            else:
                converted_weight = converted_weight.permute(3, 2, 0, 1)

        assert torch_weights[torch_k].shape == converted_weight.shape, f"shape mismatch: {k}"

        torch_converted_weights[torch_k] = converted_weight

    assert sorted(list(torch_converted_weights)) == sorted(list(torch_weights)), f"some weights are missing"
    torch_net.load_state_dict(torch_converted_weights)    
    torch.save(torch_net.state_dict(), save_name)
    print(f"PyTorch model saved at {save_name}")
    

if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--tf_checkpoint_path',
        type=str,
        default='AnimeGANv2/checkpoint/generator_Paprika_weight',
    )
    parser.add_argument(
        '--save_name', 
        type=str, 
        default='pytorch_generator_Paprika.pt',
    )
    args = parser.parse_args()
    
    convert_and_save(args.tf_checkpoint_path, args.save_name)
'''
file = open("convert_weights2.py","w")
file.write(text)
file.close()

In [10]:
# test code 만들기
text = '''
import sys
sys.path.append('/content/animegan2-pytorch')

import torch
import cv2
import numpy as np
import os

from model import Generator
import easydict

torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


def load_image(image_path, x32=False):
    img = cv2.imread(image_path).astype(np.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    if x32:  # resize image to multiple of 32s
        def to_32s(x):
            return 256 if x < 256 else x - x % 32

        img = cv2.resize(img, (to_32s(w), to_32s(h)))

    img = torch.from_numpy(img)
    img = img / 127.5 - 1.0
    return img


def test(args):
    device = args.device

    net = Generator()
    net.load_state_dict(torch.load(args.checkpoint, map_location="cpu"))
    net.to(device).eval()
    print(f"model loaded: {args.checkpoint}")

    os.makedirs(args.output_dir, exist_ok=True)

    for image_name in sorted(os.listdir(args.input_dir)):
        if os.path.splitext(image_name)[-1].lower() not in [".jpg", ".png", ".bmp", ".tiff"]:
            continue

        image = load_image(os.path.join(args.input_dir, image_name), args.x32)

        with torch.no_grad():
            input = image.permute(2, 0, 1).unsqueeze(0).to(device)
            out = net(input, args.upsample_align).squeeze(0).permute(1, 2, 0).cpu().numpy()
            out = (out + 1) * 127.5
            out = np.clip(out, 0, 255).astype(np.uint8)

        cv2.imwrite(os.path.join(args.output_dir, image_name), cv2.cvtColor(out, cv2.COLOR_BGR2RGB))
        print(f"image saved: {image_name}")


if __name__ == '__main__':

    args = easydict.EasyDict({

        "checkpoint": '/content/pytorch_generator_Hayao.pt',

        "input_dir": '/content/static/inputs',

        "output_dir": '/content/static/anime_outputs',

        "device": 'cpu',

        "upsample_align": False,

        "x32": "store_true"

    })


    test(args)

'''
file = open("anime_execute.py","w")
file.write(text)
file.close()

In [11]:
%cd /content
! python /content/animegan2-pytorch/convert_weights2.py --tf_checkpoint_path '/content/AnimeGANv2/checkpoint/generator_Hayao_weight' --save_name 'pytorch_generator_Hayao.pt'

/content


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.





2021-06-08 13:47:38.392591: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-06-08 13:47:38.399594: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000175000 Hz
2021-06-08 13:47:38.399972: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b9a4cc7480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-06-08 13:47:38.400009: I tensorflow/compiler/xla/servi

#Pose Transfer Tools

In [12]:
%cd content

!git clone https://github.com/tengteng95/Pose-Transfer.git
%cd Pose-Transfer


[Errno 2] No such file or directory: 'content'
/content
Cloning into 'Pose-Transfer'...
remote: Enumerating objects: 423, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 423 (delta 36), reused 47 (delta 23), pack-reused 342
Receiving objects: 100% (423/423), 15.80 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (202/202), done.
/content/Pose-Transfer


In [13]:
%cd ./fashion_data


!mkdir ./test
!mkdir ./train
!mkdir ./testK



/content/Pose-Transfer/fashion_data


In [14]:
import gdown

url_1 = 'https://drive.google.com/u/0/uc?id=1D1Mb6EEfMgwRXgyxho8qsENkP6U08e04'
annotation = 'img.zip'
gdown.download(url_1,annotation,quiet = False)



Downloading...
From: https://drive.google.com/u/0/uc?id=1D1Mb6EEfMgwRXgyxho8qsENkP6U08e04
To: /content/Pose-Transfer/fashion_data/img.zip
831MB [00:07, 117MB/s]


'img.zip'

In [15]:
!unzip -qq './img.zip'

In [16]:
%cd /content/Pose-Transfer/tool

text = '''
import os
import shutil
from PIL import Image

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)


def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir
    new_root = './fashion_data'
    if not os.path.exists(new_root):
        os.mkdir(new_root)

    train_root = './fashion_data/train'
    if not os.path.exists(train_root):
        os.mkdir(train_root)

    test_root = './fashion_data/test'
    if not os.path.exists(test_root):
        os.mkdir(test_root)

    train_images = []
    train_f = open('./fashion_data/train.lst', 'r')
    for lines in train_f:
        lines = lines.strip()
        if lines.endswith('.jpg'):
            train_images.append(lines)

    test_images = []
    test_f = open('./fashion_data/test.lst', 'r')
    for lines in test_f:
        lines = lines.strip()
        if lines.endswith('.jpg'):
            test_images.append(lines)

    print(train_images, test_images)

    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            if is_image_file(fname):
                path = os.path.join(root, fname)
                path_names = path.split('/')
                #print(path_names)
                path_names[0] = ""
                path_names[1] = ""
                path_names[2] = ""
                path_names[3] = ""
                path_names[4] = "fashion"
                path_names[5] = path_names[5].replace('_', '')
                # print(path_names[6])
                # path_names[6] = path_names[6].split('_')[0]  + "".join(path_names[6].split('_')[1:])
                path_names[7] = path_names[7].split('_')[0] + "".join(path_names[7].split('_')[1:])
                path_names[8] = path_names[8].split('_')[0] + '_' + "".join(path_names[8].split('_')[1:])
                path_names = "".join(path_names)
                # new_path = os.path.join(root, path_names)
                img = Image.open(path)
                imgcrop = img.crop((40, 0, 216, 256))
                if path_names in train_images:
                    imgcrop.save(os.path.join(train_root, path_names))
                elif path_names in test_images:
                    imgcrop.save(os.path.join(test_root, path_names))
                    print("in!")
                    print(os.path.join(test_root, path_names))


make_dataset('/content/Pose-Transfer')
'''
file = open("generate_fashion_datasets.py","w")
file.write(text)
file.close()

/content/Pose-Transfer/tool


In [17]:
%cd /content/Pose-Transfer

!python tool/generate_fashion_datasets.py

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
in!
./fashion_data/test/fashionWOMENDressesid0000234804_4full.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000234804_2side.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000234804_1front.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000243402_2side.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000243402_4full.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000243402_1front.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000243402_3back.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000243402_7additional.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000251701_3back.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000251701_7additional.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000251701_2side.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000251701_4full.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000251701_1front.jpg
in!
./fashion_data/test/fashionWOMENDressesid0000257502_2side.jpg
in!
./fashion_data/test/f

In [18]:

%cd /content/Pose-Transfer/fashion_data

url_1 = 'https://drive.google.com/u/0/uc?id=1g-_JS40wbROfPdOkoVRJpDn0q6D_2trX'
annotation = 'fashion-resize-annotation-test.csv'
gdown.download(url_1,annotation,quiet = False)

/content/Pose-Transfer/fashion_data


Downloading...
From: https://drive.google.com/u/0/uc?id=1g-_JS40wbROfPdOkoVRJpDn0q6D_2trX
To: /content/Pose-Transfer/fashion_data/fashion-resize-annotation-test.csv
100%|██████████| 812k/812k [00:00<00:00, 89.5MB/s]


'fashion-resize-annotation-test.csv'

In [65]:

%cd /content/Pose-Transfer/tool/

text = '''
import numpy as np
import pandas as pd 
import json
import os 

MISSING_VALUE = -1
# fix PATH
img_dir  = './fashion_data' #raw image path
annotations_file = '/content/Pose-Transfer/fashion_data/fashion-resize-annotation-test.csv' #pose annotation path
save_path = '/content/Pose-Transfer/fashion_data/testK' #path to store pose maps

def load_pose_cords_from_strings(y_str, x_str):
    y_cords = json.loads(y_str)
    x_cords = json.loads(x_str)
    return np.concatenate([np.expand_dims(y_cords, -1), np.expand_dims(x_cords, -1)], axis=1)

def cords_to_map(cords, img_size, sigma=6):
    result = np.zeros(img_size + cords.shape[0:1], dtype='uint8')
    for i, point in enumerate(cords):
        if point[0] == MISSING_VALUE or point[1] == MISSING_VALUE:
            continue
        xx, yy = np.meshgrid(np.arange(img_size[1]), np.arange(img_size[0]))
        result[..., i] = np.exp(-((yy - point[0]) ** 2 + (xx - point[1]) ** 2) / (2 * sigma ** 2))
        # result[..., i] = np.where(((yy - point[0]) ** 2 + (xx - point[1]) ** 2) < (sigma ** 2), 1, 0)
    return result

def compute_pose(image_dir, annotations_file, savePath, sigma):
    annotations_file = pd.read_csv(annotations_file, sep=':')
    annotations_file = annotations_file.set_index('name')
    image_size = (256, 176)
    cnt = len(annotations_file)
    for i in range(cnt):
        print('processing %d / %d ...' %(i, cnt))
        row = annotations_file.iloc[i]
        name = row.name
        print(savePath, name)
        file_name = os.path.join(savePath, name + '.npy')
        kp_array = load_pose_cords_from_strings(row.keypoints_y, row.keypoints_x)
        pose = cords_to_map(kp_array, image_size, sigma)
        np.save(file_name, pose)
        # input()
  
compute_pose(img_dir, annotations_file, save_path,6)




'''
file = open("generate_pose_map_fashion.py","w")
file.write(text)
file.close()

/content/Pose-Transfer/tool


In [87]:
#################얘는 input data 바꿔줄거임
%cd /content/Pose-Transfer/tool/

text = '''
import numpy as np
import pandas as pd 
import json
import os 

MISSING_VALUE = -1
# fix PATH
img_dir  = './fashion_data' #raw image path
annotations_file = '/content/Pose-Transfer/fashion_data/inut-resize-annotation.csv' 
save_path = '/content/Pose-Transfer/fashion_data/testK' 

def load_pose_cords_from_strings(y_str, x_str):
    y_cords = json.loads(y_str)
    x_cords = json.loads(x_str)
    return np.concatenate([np.expand_dims(y_cords, -1), np.expand_dims(x_cords, -1)], axis=1)

def cords_to_map(cords, img_size, sigma=6):
    result = np.zeros(img_size + cords.shape[0:1], dtype='uint8')
    for i, point in enumerate(cords):
        if point[0] == MISSING_VALUE or point[1] == MISSING_VALUE:
            continue
        xx, yy = np.meshgrid(np.arange(img_size[1]), np.arange(img_size[0]))
        result[..., i] = np.exp(-((yy - point[0]) ** 2 + (xx - point[1]) ** 2) / (2 * sigma ** 2))
        # result[..., i] = np.where(((yy - point[0]) ** 2 + (xx - point[1]) ** 2) < (sigma ** 2), 1, 0)
    return result

def compute_pose(image_dir, annotations_file, savePath, sigma):
    annotations_file = pd.read_csv(annotations_file, sep=':')
    annotations_file = annotations_file.set_index('name')
    image_size = (256, 176)
    cnt = len(annotations_file)
    for i in range(cnt):
        print('processing %d / %d ...' %(i, cnt))
        row = annotations_file.iloc[i]
        name = row.name
        print(savePath, name)
        file_name = os.path.join(savePath, name + '.npy')
        kp_array = load_pose_cords_from_strings(row.keypoints_y, row.keypoints_x)
        pose = cords_to_map(kp_array, image_size, sigma)
        np.save(file_name, pose)
        # input()
  
compute_pose(img_dir, annotations_file, save_path,6)




'''
file = open("generate_pose_map_dataset.py","w")
file.write(text)
file.close()

/content/Pose-Transfer/tool


In [21]:
%cd /content/Pose-Transfer/

!python tool/generate_pose_map_fashion.py

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
processing 1538 / 4038 ...
/content/Pose-Transfer/fashion_data/testK fashionWOMENTees_Tanksid0000634205_3back.jpg
processing 1539 / 4038 ...
/content/Pose-Transfer/fashion_data/testK fashionWOMENSweatersid0000029104_2side.jpg
processing 1540 / 4038 ...
/content/Pose-Transfer/fashion_data/testK fashionWOMENSweatersid0000347906_2side.jpg
processing 1541 / 4038 ...
/content/Pose-Transfer/fashion_data/testK fashionWOMENBlouses_Shirtsid0000427404_4full.jpg
processing 1542 / 4038 ...
/content/Pose-Transfer/fashion_data/testK fashionWOMENTees_Tanksid0000476801_1front.jpg
processing 1543 / 4038 ...
/content/Pose-Transfer/fashion_data/testK fashionWOMENPantsid0000073002_1front.jpg
processing 1544 / 4038 ...
/content/Pose-Transfer/fashion_data/testK fashionMENTees_Tanksid0000177427_3back.jpg
processing 1545 / 4038 ...
/content/Pose-Transfer/fashion_data/testK fashionMENShirts_Polosid0000154003_1front.jpg
processing 1546 / 4038 ...
/content/Pose-Transfer/fashio

In [22]:
%cd /content/Pose-Transfer

!mkdir './checkpoints'
!mkdir './checkpoints/fashion_PATN/'

url = 'https://drive.google.com/u/0/uc?id=11rGoYlMmj1WK-qdQU4rL_5j8x7NBKjuA'

output = '/content/Pose-Transfer/checkpoints/fashion_PATN/latest_net_netG.pth'

gdown.download(url, output, quiet=False)

/content/Pose-Transfer


Downloading...
From: https://drive.google.com/u/0/uc?id=11rGoYlMmj1WK-qdQU4rL_5j8x7NBKjuA
To: /content/Pose-Transfer/checkpoints/fashion_PATN/latest_net_netG.pth
166MB [00:00, 184MB/s]


'/content/Pose-Transfer/checkpoints/fashion_PATN/latest_net_netG.pth'

In [23]:
url = 'https://drive.google.com/u/0/uc?id=18Ap45JXcGsyrRNKWO2Ec70T_-xzR1jsW'

output = '/content/Pose-Transfer/checkpoints/fashion_PATN/latest_net_netD_PP.pth'

gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/u/0/uc?id=18Ap45JXcGsyrRNKWO2Ec70T_-xzR1jsW
To: /content/Pose-Transfer/checkpoints/fashion_PATN/latest_net_netD_PP.pth
15.7MB [00:00, 104MB/s] 


'/content/Pose-Transfer/checkpoints/fashion_PATN/latest_net_netD_PP.pth'

In [24]:
url = 'https://drive.google.com/u/0/uc?id=1Rlw4tlqx6Owf7uEf0IN-hMQ_v5dKoKC8'

output = '/content/Pose-Transfer/checkpoints/fashion_PATN/latest_net_netD_PB.pth'

gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/u/0/uc?id=1Rlw4tlqx6Owf7uEf0IN-hMQ_v5dKoKC8
To: /content/Pose-Transfer/checkpoints/fashion_PATN/latest_net_netD_PB.pth
15.9MB [00:00, 171MB/s]


'/content/Pose-Transfer/checkpoints/fashion_PATN/latest_net_netD_PB.pth'

In [25]:
import torch

ckp_path = './checkpoints/fashion_PATN/latest_net_netG.pth'
save_path = './checkpoints/fashion_PATN/latest_net_netG.pth'
states_dict = torch.load(ckp_path)
states_dict_new = states_dict.copy()
for key in states_dict.keys():
	if "running_var" in key or "running_mean" in key:
		del states_dict_new[key]

torch.save(states_dict_new, save_path)

In [26]:
%cd /content/Pose-Transfer/
url_1 = 'https://drive.google.com/u/0/uc?id=1nOC5YOOUZEdqbOdmV9Tbu6soKy4Mwke4'
annotation = 'pose_estimator.h5'
gdown.download(url_1,annotation,quiet = False)

/content/Pose-Transfer


Downloading...
From: https://drive.google.com/u/0/uc?id=1nOC5YOOUZEdqbOdmV9Tbu6soKy4Mwke4
To: /content/Pose-Transfer/pose_estimator.h5
210MB [00:00, 229MB/s]


'pose_estimator.h5'

In [86]:
%cd /content/Pose-Transfer/tool

text = '''
import pose_utils
import os
import numpy as np
import imp
import sys

cmd = imp.load_source("cmd", "/usr/lib/python2.7/cmd.py")

from keras.models import load_model
import skimage.transform as st
import pandas as pd
from tqdm import tqdm
from skimage.io import imread
from skimage.transform import resize
from scipy.ndimage import gaussian_filter



model = load_model("/content/Pose-Transfer/pose_estimator.h5")


mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], [19,20], [21,22],
          [23,24], [25,26], [27,28], [29,30], [47,48], [49,50], [53,54], [51,52],
          [55,56], [37,38], [45,46]]

limbSeq = [[2,3], [2,6], [3,4], [4,5], [6,7], [7,8], [2,9], [9,10],
           [10,11], [2,12], [12,13], [13,14], [2,1], [1,15], [15,17],
           [1,16], [16,18], [3,17], [6,18]]


def compute_cordinates(heatmap_avg, paf_avg, th1=0.1, th2=0.05):
    all_peaks = []
    peak_counter = 0

    for part in range(18):
        map_ori = heatmap_avg[:,:,part]
        map = gaussian_filter(map_ori, sigma=3)

        map_left = np.zeros(map.shape)
        map_left[1:,:] = map[:-1,:]
        map_right = np.zeros(map.shape)
        map_right[:-1,:] = map[1:,:]
        map_up = np.zeros(map.shape)
        map_up[:,1:] = map[:,:-1]
        map_down = np.zeros(map.shape)
        map_down[:,:-1] = map[:,1:]

        peaks_binary = np.logical_and.reduce((map>=map_left, map>=map_right, map>=map_up, map>=map_down, map > th1))
        peaks = zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0]) # note reverse

        peaks_with_score = [x + (map_ori[x[1],x[0]],) for x in peaks]
        id = range(peak_counter, peak_counter + len(peaks))
        peaks_with_score_and_id = [peaks_with_score[i] + (id[i],) for i in range(len(id))]

        all_peaks.append(peaks_with_score_and_id)
        peak_counter += len(peaks)

    connection_all = []
    special_k = []
    mid_num = 10

    for k in range(len(mapIdx)):
        score_mid = paf_avg[:,:,[x-19 for x in mapIdx[k]]]
        candA = all_peaks[limbSeq[k][0]-1]
        candB = all_peaks[limbSeq[k][1]-1]
        nA = len(candA)
        nB = len(candB)
        indexA, indexB = limbSeq[k]
        if(nA != 0 and nB != 0):
            connection_candidate = []
            for i in range(nA):
                for j in range(nB):
                    vec = np.subtract(candB[j][:2], candA[i][:2])
                    norm = np.sqrt(vec[0]*vec[0] + vec[1]*vec[1])
                    vec = np.divide(vec, norm)

                    startend = zip(np.linspace(candA[i][0], candB[j][0], num=mid_num),
                                   np.linspace(candA[i][1], candB[j][1], num=mid_num))

                    vec_x = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 0]
                                      for I in range(len(startend))])
                    vec_y = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 1]
                                      for I in range(len(startend))])

                    score_midpts = np.multiply(vec_x, vec[0]) + np.multiply(vec_y, vec[1])
                    score_with_dist_prior = sum(score_midpts)/len(score_midpts) + min(0.5*oriImg.shape[0]/norm-1, 0)
                    criterion1 = len(np.nonzero(score_midpts > th2)[0]) > 0.8 * len(score_midpts)
                    criterion2 = score_with_dist_prior > 0
                    if criterion1 and criterion2:
                        connection_candidate.append([i, j, score_with_dist_prior, score_with_dist_prior+candA[i][2]+candB[j][2]])

            connection_candidate = sorted(connection_candidate, key=lambda x: x[2], reverse=True)
            connection = np.zeros((0,5))
            for c in range(len(connection_candidate)):
                i,j,s = connection_candidate[c][0:3]
                if(i not in connection[:,3] and j not in connection[:,4]):
                    connection = np.vstack([connection, [candA[i][3], candB[j][3], s, i, j]])
                    if(len(connection) >= min(nA, nB)):
                        break

            connection_all.append(connection)
        else:
            special_k.append(k)
            connection_all.append([])

    # last number in each row is the total parts number of that person
    # the second last number in each row is the score of the overall configuration
    subset = -1 * np.ones((0, 20))
    candidate = np.array([item for sublist in all_peaks for item in sublist])

    for k in range(len(mapIdx)):
        if k not in special_k:
            partAs = connection_all[k][:,0]
            partBs = connection_all[k][:,1]
            indexA, indexB = np.array(limbSeq[k]) - 1

            for i in range(len(connection_all[k])): #= 1:size(temp,1)
                found = 0
                subset_idx = [-1, -1]
                for j in range(len(subset)): #1:size(subset,1):
                    if subset[j][indexA] == partAs[i] or subset[j][indexB] == partBs[i]:
                        subset_idx[found] = j
                        found += 1

                if found == 1:
                    j = subset_idx[0]
                    if(subset[j][indexB] != partBs[i]):
                        subset[j][indexB] = partBs[i]
                        subset[j][-1] += 1
                        subset[j][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]
                elif found == 2: # if found 2 and disjoint, merge them
                    j1, j2 = subset_idx
                    print("found = 2")
                    # print("found = 2")
                    membership = ((subset[j1]>=0).astype(int) + (subset[j2]>=0).astype(int))[:-2]
                    if len(np.nonzero(membership == 2)[0]) == 0: #merge
                        subset[j1][:-2] += (subset[j2][:-2] + 1)
                        subset[j1][-2:] += subset[j2][-2:]
                        subset[j1][-2] += connection_all[k][i][2]
                        subset = np.delete(subset, j2, 0)
                    else: # as like found == 1
                        subset[j1][indexB] = partBs[i]
                        subset[j1][-1] += 1
                        subset[j1][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]

                # if find no partA in the subset, create a new subset
                elif not found and k < 17:
                    row = -1 * np.ones(20)
                    row[indexA] = partAs[i]
                    row[indexB] = partBs[i]
                    row[-1] = 2
                    row[-2] = sum(candidate[connection_all[k][i,:2].astype(int), 2]) + connection_all[k][i][2]
                    subset = np.vstack([subset, row])

    # delete some rows of subset which has few parts occur
    deleteIdx = [];
    for i in range(len(subset)):
        if subset[i][-1] < 4 or subset[i][-2]/subset[i][-1] < 0.4:
            deleteIdx.append(i)
    subset = np.delete(subset, deleteIdx, axis=0)

    if len(subset) == 0:
        return np.array([[-1, -1]] * 18).astype(int)

    cordinates = []
    result_image_index = np.argmax(subset[:, -2])

    for part in subset[result_image_index, :18]:
        if part == -1:
            cordinates.append([-1, -1])
        else:
            Y = candidate[part.astype(int), 0]
            X = candidate[part.astype(int), 1]
            cordinates.append([X, Y])
    return np.array(cordinates).astype(int)

image_name = '/content/static/inputs/human.jpg' # input file
output_path = '/content/Pose-Transfer/fashion_data/inut-resize-annotation.csv'

# input_folder = './results/market_PATN/test_latest/images_crop/'
# output_path = './results/test_keypoints/pckh.csv'


threshold = 0.1
boxsize = 368
scale_search = [0.5, 1, 1.5, 2]

if os.path.exists(output_path):
    processed_names = set(pd.read_csv(output_path, sep=':')['name'])
    result_file = open(output_path, 'a')
else:
    result_file = open(output_path, 'w')
    processed_names = set()
    print >> result_file, 'name:keypoints_y:keypoints_x'

# for image_name in tqdm(os.listdir(input_folder)):
if image_name not in processed_names:

    oriImg = imread(image_name)[:, :, ::-1]  # B,G,R order

    multiplier = [x * boxsize / oriImg.shape[0] for x in scale_search]

    heatmap_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 19))
    paf_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 38))

    for m in range(len(multiplier)):
        scale = multiplier[m]

        new_size = (np.array(oriImg.shape[:2]) * scale).astype(np.int32)
        imageToTest = resize(oriImg, new_size, order=3, preserve_range=True)
        imageToTest_padded = imageToTest[np.newaxis, :, :, :]/255 - 0.5

        output1, output2 = model.predict(imageToTest_padded)

        heatmap = st.resize(output2[0], oriImg.shape[:2], preserve_range=True, order=1)
        paf = st.resize(output1[0], oriImg.shape[:2], preserve_range=True, order=1)
        heatmap_avg += heatmap
        paf_avg += paf

    heatmap_avg /= len(multiplier)

    pose_cords = compute_cordinates(heatmap_avg, paf_avg)

    print >> result_file, "%s: %s: %s" % ('human.jpg', str(list(pose_cords[:, 0])), str(list(pose_cords[:, 1])))
    result_file.flush()

    '''
file = open("compute_coordinates.py","w")
file.write(text)
file.close()


/content/Pose-Transfer/tool


In [28]:
%cd /content/Pose-Transfer/options/

text = '''
import argparse
import os
from util import util
import torch


class BaseOptions():
    def __init__(self):
        self.parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
        self.initialized = False

    def initialize(self):
        self.parser.add_argument('--dataroot', default='./fashion_data', help='path to images (should have subfolders trainA, trainB, valA, valB, etc)')
        self.parser.add_argument('--batchSize', type=int, default=1, help='input batch size')
        self.parser.add_argument('--loadSize', type=int, default=286, help='scale images to this size')
        self.parser.add_argument('--fineSize', type=int, default=256, help='then crop to this size')
        self.parser.add_argument('--input_nc', type=int, default=3, help='# of input image channels')
        self.parser.add_argument('--output_nc', type=int, default=3, help='# of output image channels')
        self.parser.add_argument('--ngf', type=int, default=64, help='# of gen filters in first conv layer')
        self.parser.add_argument('--ndf', type=int, default=64, help='# of discrim filters in first conv layer')
        self.parser.add_argument('--which_model_netD', type=str, default='resnet', help='selects model to use for netD')
        self.parser.add_argument('--which_model_netG', type=str, default='PATN', help='selects model to use for netG')
        self.parser.add_argument('--n_layers_D', type=int, default=3, help='blocks used in D')
        self.parser.add_argument('--gpu_ids', type=str, default='0', help='gpu ids: e.g. 0  0,1,2, 0,2. use -1 for CPU')
        self.parser.add_argument('--name', type=str, default='fashion_PATN', help='name of the experiment. It decides where to store samples and models')
        self.parser.add_argument('--dataset_mode', type=str, default='keypoint', help='chooses how datasets are loaded. [unaligned | aligned | single]')
        self.parser.add_argument('--model', type=str, default='PATN',
                                 help='chooses which model to use. cycle_gan, pix2pix, test')
        self.parser.add_argument('--which_direction', type=str, default='AtoB', help='AtoB or BtoA')
        self.parser.add_argument('--nThreads', default=2, type=int, help='# threads for loading data')
        self.parser.add_argument('--checkpoints_dir', type=str, default='./checkpoints', help='models are saved here')
        self.parser.add_argument('--norm', type=str, default='instance', help='instance normalization or batch normalization')
        self.parser.add_argument('--serial_batches', action='store_true', help='if true, takes images in order to make batches, otherwise takes them randomly')
        self.parser.add_argument('--display_winsize', type=int, default=256,  help='display window size')
        self.parser.add_argument('--display_id', type=int, default=0, help='window id of the web display')
        self.parser.add_argument('--display_port', type=int, default=8097, help='visdom port of the web display')
        self.parser.add_argument('--no_dropout', action='store_true', help='no dropout for the generator')
        self.parser.add_argument('--max_dataset_size', type=int, default=float("inf"), help='Maximum number of samples allowed per dataset. If the dataset directory contains more than max_dataset_size, only a subset is loaded.')
        self.parser.add_argument('--resize_or_crop', type=str, default='resize_and_crop', help='scaling and cropping of images at load time [resize_and_crop|crop|scale_width|scale_width_and_crop]')
        self.parser.add_argument('--no_flip', action='store_true',default=True, help='if specified, do not flip the images for data augmentation')
        self.parser.add_argument('--init_type', type=str, default='normal', help='network initialization [normal|xavier|kaiming|orthogonal]')

        self.parser.add_argument('--P_input_nc', type=int, default=3, help='# of input image channels')
        self.parser.add_argument('--BP_input_nc', type=int, default=18, help='# of input image channels')
        self.parser.add_argument('--padding_type', type=str, default='reflect', help='# of input image channels')
        self.parser.add_argument('--pairLst', type=str, default='./fashion_data/fashion-resize-pairsptest.csv', help='market pairs')

        self.parser.add_argument('--with_D_PP', type=int, default=1, help='use D to judge P and P is pair or not')
        self.parser.add_argument('--with_D_PB', type=int, default=1, help='use D to judge P and B is pair or not')

        self.parser.add_argument('--use_flip', type=int, default=0, help='flip or not')

        # down-sampling times
        self.parser.add_argument('--G_n_downsampling', type=int, default=2, help='down-sampling blocks for generator')
        self.parser.add_argument('--D_n_downsampling', type=int, default=2, help='down-sampling blocks for discriminator')

        self.initialized = True

    def parse(self):
        if not self.initialized:
            self.initialize()
        self.opt = self.parser.parse_args()
        self.opt.isTrain = self.isTrain   # train or test

        str_ids = self.opt.gpu_ids.split(',')
        self.opt.gpu_ids = []
        for str_id in str_ids:
            id = int(str_id)
            if id >= 0:
                self.opt.gpu_ids.append(id)

        # set gpu ids
        if len(self.opt.gpu_ids) > 0:
            torch.cuda.set_device(self.opt.gpu_ids[0])

        args = vars(self.opt)

        print('------------ Options -------------')
        for k, v in sorted(args.items()):
            print('%s: %s' % (str(k), str(v)))
        print('-------------- End ----------------')

        # save to the disk
        expr_dir = os.path.join(self.opt.checkpoints_dir, self.opt.name)
        util.mkdirs(expr_dir)
        file_name = os.path.join(expr_dir, 'opt.txt')
        with open(file_name, 'wt') as opt_file:
            opt_file.write('------------ Options -------------\n')
            for k, v in sorted(args.items()):
                opt_file.write('%s: %s\n' % (str(k), str(v)))
            opt_file.write('-------------- End ----------------\n')
        return self.opt
        '''
file = open("base_options.py","w")
file.write(text)
file.close()


  

/content/Pose-Transfer/options


#Pose Transfer function(resize, compute-posemap, make-train-pair)

In [81]:
from PIL import Image
import os
import csv


def resizing_and_pose_map(imgpath,imgnames):
  img = Image.open(imgpath)
  imgcrop = img.crop((40, 0, 216, 256))
  imgcrop.save(os.path.join('/content/Pose-Transfer/fashion_data/test/',imgnames))

  try:
    subprocess.call(['python2', '/content/Pose-Transfer/tool/compute_coordinates.py'])
    print('ok!')
  except:
    print('error#####################################')

  execfile('/content/Pose-Transfer/tool/generate_pose_map_dataset.py')

def make_pair(select, imgnames):
  f = open('temp.csv', 'w', encoding = 'utf-8', newline='')
  wr = csv.writer(f)
  
  i=int(select)-1
  #1:뒤 2:정면(시연 사진) 3:left 4 right 5 walk
  match_pose = ['fashionWOMENBlouses_Shirtsid0000073702_3back',
                'fashionWOMENBlouses_Shirtsid0000388202_1front',
                'fashionWOMENDressesid0000455102_3back',
                'fashionWOMENBlouses_Shirtsid0000001502_4full',
                'fashionWOMENDressesid0000573204_4full'
                ]

  wr.writerow(['from', 'to'])
  wr.writerow([imgnames, match_pose[i]])
  f.close()


In [94]:
def test_pose():
    %cd /content/Pose-Transfer
    execfile('/content/Pose-Transfer/test.py')

# paprika version

In [30]:
%cd /content
%cd animegan2-pytorch

/content
/content/animegan2-pytorch


In [31]:
# test code 만들기
text = '''
import sys
sys.path.append('/content/animegan2-pytorch')

import torch
import cv2
import numpy as np
import os

from model import Generator
import easydict

torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


def load_image(image_path, x32=False):
    img = cv2.imread(image_path).astype(np.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    if x32:  # resize image to multiple of 32s
        def to_32s(x):
            return 256 if x < 256 else x - x % 32

        img = cv2.resize(img, (to_32s(w), to_32s(h)))

    img = torch.from_numpy(img)
    img = img / 127.5 - 1.0
    return img


def test(args):
    device = args.device

    net = Generator()
    net.load_state_dict(torch.load(args.checkpoint, map_location="cpu"))
    net.to(device).eval()
    print(f"model loaded: {args.checkpoint}")

    os.makedirs(args.output_dir, exist_ok=True)

    for image_name in sorted(os.listdir(args.input_dir)):
        if os.path.splitext(image_name)[-1].lower() not in [".jpg", ".png", ".bmp", ".tiff"]:
            continue

        image = load_image(os.path.join(args.input_dir, image_name), args.x32)

        with torch.no_grad():
            input = image.permute(2, 0, 1).unsqueeze(0).to(device)
            out = net(input, args.upsample_align).squeeze(0).permute(1, 2, 0).cpu().numpy()
            out = (out + 1) * 127.5
            out = np.clip(out, 0, 255).astype(np.uint8)

        cv2.imwrite(os.path.join(args.output_dir, image_name), cv2.cvtColor(out, cv2.COLOR_BGR2RGB))
        print(f"image saved: {image_name}")


if __name__ == '__main__':

    args = easydict.EasyDict({

        "checkpoint": '/content/pytorch_generator_Paprika.pt',

        "input_dir": '/content/static/inputs',

        "output_dir": '/content/static/anime_outputs',

        "device": 'cpu',

        "upsample_align": False,

        "x32": "store_true"

    })


    test(args)

'''
file = open("anime_execute2.py","w")
file.write(text)
file.close()

In [32]:
%cd /content
! python /content/animegan2-pytorch/convert_weights2.py 

/content


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.





2021-06-08 13:52:46.007870: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-06-08 13:52:46.014019: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000175000 Hz
2021-06-08 13:52:46.014392: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556329017480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-06-08 13:52:46.014445: I tensorflow/compiler/xla/servi

#인물사진 segmentation

In [33]:
%cd /content
%cd Self-Correction-Human-Parsing

/content
/content/Self-Correction-Human-Parsing


In [34]:
dataset = 'atr'         #select from ['lip', 'atr', 'pascal']

if dataset == 'lip':
    url = 'https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH'
elif dataset == 'atr':
    url = 'https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP'
elif dataset == 'pascal':
    url = 'https://drive.google.com/uc?id=1E5YwNKW2VOEayK9mWCS3Kpsxf-3z04ZE'

output = 'checkpoints/final.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP
To: /content/Self-Correction-Human-Parsing/checkpoints/final.pth
267MB [00:02, 118MB/s]


'checkpoints/final.pth'

In [35]:
text = '''

#!/usr/bin/env python
# -*- encoding: utf-8 -*-

import os
import torch
import easydict
import numpy as np
from PIL import Image
from tqdm import tqdm

from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import networks
from utils.transforms import transform_logits
from datasets.simple_extractor_dataset import SimpleFolderDataset

dataset_settings = {
    'lip': {
        'input_size': [473, 473],
        'num_classes': 20,
        'label': ['Background', 'Hat', 'Hair', 'Glove', 'Sunglasses', 'Upper-clothes', 'Dress', 'Coat',
                  'Socks', 'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-arm', 'Right-arm',
                  'Left-leg', 'Right-leg', 'Left-shoe', 'Right-shoe']
    },
    'atr': {
        'input_size': [512, 512],
        'num_classes': 18,
        'label': ['Background', 'Hat', 'Hair', 'Sunglasses', 'Upper-clothes', 'Skirt', 'Pants', 'Dress', 'Belt',
                  'Left-shoe', 'Right-shoe', 'Face', 'Left-leg', 'Right-leg', 'Left-arm', 'Right-arm', 'Bag', 'Scarf']
    },
    'pascal': {
        'input_size': [512, 512],
        'num_classes': 7,
        'label': ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs'],
    }
}


def get_arguments():
    """Parse all the arguments provided from the CLI.
    Returns:
      A list of parsed arguments.
    """
    args = easydict.EasyDict({
        "dataset": 'atr',
        "model_restore": 'checkpoints/final.pth',
        "gpu": '0',
        "input_dir": 'inputs',
        "output_dir": '/content/static/seg_outputs',
        "logits": False,
    })
    return args


def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette


def main():
    args = get_arguments()

    gpus = [int(i) for i in args.gpu.split(',')]
    assert len(gpus) == 1
    if not args.gpu == 'None':
        os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

    num_classes = dataset_settings[args.dataset]['num_classes']
    input_size = dataset_settings[args.dataset]['input_size']
    label = dataset_settings[args.dataset]['label']
    print("Evaluating total class number {} with {}".format(num_classes, label))

    model = networks.init_model('resnet101', num_classes=num_classes, pretrained=None)

    state_dict = torch.load(args.model_restore)['state_dict']
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:]  # remove `module.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    model.cuda()
    model.eval()

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])
    ])
    dataset = SimpleFolderDataset(root=args.input_dir, input_size=input_size, transform=transform)
    dataloader = DataLoader(dataset)

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    palette = [255, 255, 255,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0, ]
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(dataloader)):
            image, meta = batch
            img_name = meta['name'][0]
            c = meta['center'].numpy()[0]
            s = meta['scale'].numpy()[0]
            w = meta['width'].numpy()[0]
            h = meta['height'].numpy()[0]

            output = model(image.cuda())
            upsample = torch.nn.Upsample(size=input_size, mode='bilinear', align_corners=True)
            upsample_output = upsample(output[0][-1][0].unsqueeze(0))
            upsample_output = upsample_output.squeeze()
            upsample_output = upsample_output.permute(1, 2, 0)  # CHW -> HWC

            logits_result = transform_logits(upsample_output.data.cpu().numpy(), c, s, w, h, input_size=input_size)
            parsing_result = np.argmax(logits_result, axis=2)
            parsing_result_path = os.path.join(args.output_dir, img_name[:-4] + '.png')
            output_img = Image.fromarray(np.asarray(parsing_result, dtype=np.uint8))
            output_img.putpalette(palette)
            output_img.save(parsing_result_path)
            if args.logits:
                logits_result_path = os.path.join(args.output_dir, img_name[:-4] + '.npy')
                np.save(logits_result_path, logits_result)
    return


if __name__ == '__main__':
    main()


'''
file = open("simple_extractor_edit_b.py","w")
file.write(text)
file.close()

In [36]:
text = '''

#!/usr/bin/env python
# -*- encoding: utf-8 -*-

import sys
sys.path.append('/content/Self-Correction-Human-Parsing')

import os
import torch
import easydict
import numpy as np
from PIL import Image
from tqdm import tqdm

from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import networks
from utils.transforms import transform_logits
from datasets.simple_extractor_dataset import SimpleFolderDataset

dataset_settings = {
    'lip': {
        'input_size': [473, 473],
        'num_classes': 20,
        'label': ['Background', 'Hat', 'Hair', 'Glove', 'Sunglasses', 'Upper-clothes', 'Dress', 'Coat',
                  'Socks', 'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-arm', 'Right-arm',
                  'Left-leg', 'Right-leg', 'Left-shoe', 'Right-shoe']
    },
    'atr': {
        'input_size': [512, 512],
        'num_classes': 18,
        'label': ['Background', 'Hat', 'Hair', 'Sunglasses', 'Upper-clothes', 'Skirt', 'Pants', 'Dress', 'Belt',
                  'Left-shoe', 'Right-shoe', 'Face', 'Left-leg', 'Right-leg', 'Left-arm', 'Right-arm', 'Bag', 'Scarf']
    },
    'pascal': {
        'input_size': [512, 512],
        'num_classes': 7,
        'label': ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs'],
    }
}


def get_arguments():
    """Parse all the arguments provided from the CLI.
    Returns:
      A list of parsed arguments.
    """
    args = easydict.EasyDict({
        "dataset": 'atr',
        "model_restore": 'checkpoints/final.pth',
        "gpu": '0',
        "input_dir": 'inputs',
        "output_dir": '/content/static/seg_outputs',
        "logits": False,
    })
    return args


def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette


def main():
    args = get_arguments()

    gpus = [int(i) for i in args.gpu.split(',')]
    assert len(gpus) == 1
    if not args.gpu == 'None':
        os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

    num_classes = dataset_settings[args.dataset]['num_classes']
    input_size = dataset_settings[args.dataset]['input_size']
    label = dataset_settings[args.dataset]['label']
    print("Evaluating total class number {} with {}".format(num_classes, label))

    model = networks.init_model('resnet101', num_classes=num_classes, pretrained=None)

    state_dict = torch.load(args.model_restore)['state_dict']
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:]  # remove `module.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    model.cuda()
    model.eval()

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])
    ])
    dataset = SimpleFolderDataset(root=args.input_dir, input_size=input_size, transform=transform)
    dataloader = DataLoader(dataset)

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    palette = [0, 0, 0,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255, ]
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(dataloader)):
            image, meta = batch
            img_name = meta['name'][0]
            c = meta['center'].numpy()[0]
            s = meta['scale'].numpy()[0]
            w = meta['width'].numpy()[0]
            h = meta['height'].numpy()[0]

            output = model(image.cuda())
            upsample = torch.nn.Upsample(size=input_size, mode='bilinear', align_corners=True)
            upsample_output = upsample(output[0][-1][0].unsqueeze(0))
            upsample_output = upsample_output.squeeze()
            upsample_output = upsample_output.permute(1, 2, 0)  # CHW -> HWC

            logits_result = transform_logits(upsample_output.data.cpu().numpy(), c, s, w, h, input_size=input_size)
            parsing_result = np.argmax(logits_result, axis=2)
            parsing_result_path = os.path.join(args.output_dir, img_name[:-4] + '.png')
            output_img = Image.fromarray(np.asarray(parsing_result, dtype=np.uint8))
            output_img.putpalette(palette)
            output_img.save(parsing_result_path)
            if args.logits:
                logits_result_path = os.path.join(args.output_dir, img_name[:-4] + '.npy')
                np.save(logits_result_path, logits_result)
    return


if __name__ == '__main__':
    main()

'''
file = open("simple_extractor_edit_w.py","w")
file.write(text)
file.close()

In [37]:
def segmentation(file_name):
    ## segmentation
  src = '/content/static/anime_outputs/' + file_name
  dst = '/content/Self-Correction-Human-Parsing/inputs'
  shutil.copy(src, dst)

  os.chdir('/content/Self-Correction-Human-Parsing')

  try:
    subprocess.call(['python', '/content/Self-Correction-Human-Parsing/simple_extractor_edit_b.py']) # 사람 까맣게
  except:
    print('error')

  os.rename('/content/static/seg_outputs/' + file_name.split('.')[0] + '.png', '/content/static/seg_outputs/' + file_name.split('.')[0] + '_black.png')

  try:
    subprocess.call(['python', '/content/Self-Correction-Human-Parsing/simple_extractor_edit_w.py']) # 사람 까맣게
  except:
    print('error')

  os.rename('/content/static/seg_outputs/' + file_name.split('.')[0] + '.png', '/content/static/seg_outputs/' + file_name.split('.')[0] + '_white.png')

  os.chdir('/content')
  ## finish segmentation

  return "segmentation finished"

# 이미지 전처리

In [38]:
white_img = np.full((256, 256), 255, dtype=np.uint8)
black_img = np.full((256, 256), 0, dtype=np.uint8)
color_img = np.full((256, 256, 3), (0, 255, 255), dtype=np.uint8)

In [39]:
def pre_processing(black_mask, ani_img): # 이미지 경로 말고 이미지 이름 쓰는 거 임
  src = cv2.imread('/content/static/seg_outputs/' + black_mask, cv2.IMREAD_COLOR)        # 컬러 영상
  mask = cv2.imread('/content/static/seg_outputs/' + black_mask, cv2.IMREAD_GRAYSCALE)         # 그레이스케일 영상
  dst = cv2.imread('/content/static/anime_outputs/' + ani_img, cv2.IMREAD_COLOR)

  cv2.copyTo(src, mask, dst)
  cv2.imwrite('/content/static/seg_outputs/'+ ani_img.split('.')[0] + '_white_back.jpg', dst) # 이 이미지 보여주기

  return 'preprocessing OK', '/static/seg_outputs/'+ ani_img.split('.')[0] + '_white_back.jpg'

In [40]:
def make_paddding(file_path):
  img = cv2.imread(file_path)
  a, b, c = img.shape

  differ = b - a
  differ_1, differ_2 = 0, 0

  if differ%2 == 0:
    differ_1, differ_2 = abs(differ/2), abs(differ/2)
  else:
    differ_1, differ_2 = abs(differ/2), abs(differ/2) + 1

  change_img = cv2.copyMakeBorder(img, 0, 0, int(differ_1), int(differ_2), cv2.BORDER_CONSTANT, value=[255, 255, 255])
  cv2.imwrite(file_path, change_img)

  return "make padding OK"

# 이미지 합성하기

In [41]:
def splicing_back(idx, back_file_path, human_file):
  back = cv2.imread(back_file_path, cv2.IMREAD_COLOR)        # 컬러 영상
  back_resize = cv2.resize(back, dsize=(256*4, 256*2+60), interpolation=cv2.INTER_AREA)
  #cv2_imshow(back_resize)

  sp_idx = idx
  temp = cv2.imread('/content/static/seg_outputs/' + human_file.split('.')[0] + '_white.png', cv2.IMREAD_GRAYSCALE) 
  temp2 = cv2.imread('/content/Self-Correction-Human-Parsing/inputs/'+human_file, cv2.IMREAD_COLOR)


  print(back_resize.shape, " ", temp.shape, " ", temp2.shape)



  black_4 = cv2.hconcat([black_img, black_img, black_img, black_img])
  color_4 = cv2.hconcat([color_img, color_img, color_img, color_img])

  if sp_idx == 1 or sp_idx == 5:
    black_mask = cv2.hconcat([temp, black_img, black_img, black_img])
    color_mask = cv2.hconcat([temp2, color_img, color_img, color_img])
  elif sp_idx == 2 or sp_idx == 6:
    black_mask = cv2.hconcat([black_img, temp, black_img, black_img])
    color_mask = cv2.hconcat([color_img, temp2, color_img, color_img])
  elif sp_idx == 3 or sp_idx == 7:
    black_mask = cv2.hconcat([black_img, black_img, temp, black_img])
    color_mask = cv2.hconcat([color_img, color_img, temp2, color_img])
  elif sp_idx == 4 or sp_idx == 8:
    black_mask = cv2.hconcat([black_img, black_img, black_img, temp])
    color_mask = cv2.hconcat([color_img, color_img, color_img, temp2])

  if sp_idx > 4:
    back_mask = cv2.vconcat([black_4, black_mask])
    human_mask = cv2.vconcat([color_4, color_mask])
  else:
    back_mask = cv2.vconcat([black_mask, black_4])
    human_mask = cv2.vconcat([color_mask, color_4])

  #cv2_imshow(human_mask)

  black_edge = np.full((30, 1024), 0, dtype=np.uint8)
  color_edge = np.full((30, 1024, 3), (0, 255, 255), dtype=np.uint8)

  adflkajsdf = cv2.vconcat([back_mask, black_edge])
  back_mask = cv2.vconcat([black_edge, adflkajsdf])
  #cv2_imshow(back_mask)

  adkfalkdfd = cv2.vconcat([human_mask, color_edge])
  human_mask = cv2.vconcat([color_edge, adkfalkdfd])
  #cv2_imshow(human_mask)
  #print(back_mask.shape, back_resize.shape, human_mask.shape)

  cv2.copyTo(human_mask, back_mask, back_resize)
  cv2.imwrite('/content/static/splicing_outputs/splicing_result.jpg', back_resize)

  return "finish splicing"

# 웹페이지(HTML)

In [58]:
%cd /content

/content


In [59]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
</head>

    <body>
    <p><h1 align="center">Multi-Media Computing Website</h1></p>

<div class="container">
<div class="row">
    <h2>Select a file to upload</h2>
    <h5>합성하고자 하는 배경 사진과 인물 사진을 업로드 해 주세요.</h5>


     <form action = "/fileUpload" method = "POST" enctype = "multipart/form-data">
        <dl>
            <p>
                <input type="file" name="filename[]" multiple="multiple" class="form-control" autocomplete="off" required>
            </p>
        </dl>
        <p>
        <br><br>
        <div style = "text-align:center">
            <input type="submit" value="Submit" class="btn btn-info">
            </div?
        </p>
    </form>
</div>
</div>

        </form>
    </body>
</html>
'''
file = open("templates/upload.html","w")
file.write(text)
file.close()

In [44]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
<script src="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.js"></script>
<link href="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.css" rel="stylesheet">
</head>


    <body>
    <div class="container">
        <h2 align="center"> Style Transfer Results - Animation ver</h2>
        <br><br>

  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
Backgound Image
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
Human Image
</div>
  </div>

<br>


  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src="{{data1}}" alt="background_image"  width="400">
{% endif %}
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src={{data2}} alt="human_image" width="256">
{% endif %}
</div>
</div>



</div>
<br><br><br>


<div class="container">
<div class="row">

    <form action = "/Seg_human" method = "POST">
      <input type = "hidden" name = "test2" value = {{file_name}} />        
       </select>
      </div>
      <div style = "text-align:center">
            <input type="submit" name='msg' value= "Segmentation" class="btn btn-info">
        </div>
        
    </form>

</div>
</div>

    </body>
</html>
'''

file = open("templates/showAnime.html","w")
file.write(text)
file.close()

수정중...수정중 수정중 templates/showPose.html




In [45]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
<script src="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.js"></script>
<link href="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.css" rel="stylesheet">
</head>


    <body>
    <div class="container">
        <h2 align="center"> Human Pose-Transfer-Result</h2>
        <br><br>

  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
Backgound Image
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
Human Segmentation Image
</div>
  </div>

<br>


  <div class="container; text-align:center">

<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src={{data2}} alt="human_image" width="176">
{% endif %}
</div>
</div>



</div>
<br><br><br>


<div class="container">
<div class="row">
    <h4 align="center">Select Position</h4>


    <form action = "/Select_pos" method = "POST">
      <div class="form-group" style="margin: auto; text-align:center; width: 650">
       <select name="position" id="position" class="form-control">
          <option value="">위치 선택</option>
          <option value="1">walk</option>
          <option value="2">left</option>
          <option value="3">right</option>
          <option value="4">back</option>
          <option value="5">shy</option>
       </select>
       <input type = "hidden" name = "test2" value = {{file_name}} /> 
      <Br><br><br>

            <input type="submit" value="Submit" class="btn btn-info">
        </div>
    </form>

</div>
</div>

    </body>
</html>
'''

file = open("templates/showPose.html","w")
file.write(text)
file.close()

In [46]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
<script src="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.js"></script>
<link href="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.css" rel="stylesheet">
</head>


    <body>
    <div class="container">
        <h2 align="center"> Human Segmentation Results - Animation ver</h2>
        <br><br>

  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
Backgound Image
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
Human Segmentation Image
</div>
  </div>

<br>


  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src="{{data1}}" alt="background_image"  width="400">
{% endif %}
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src={{data2}} alt="human_image" width="256">
{% endif %}
</div>
</div>



</div>
<br><br><br>


<div class="container">
<div class="row">
    <h4 align="center">Select Position</h4>


    <form action = "/Select_pos" method = "POST">
      <div class="form-group" style="margin: auto; text-align:center; width: 650">
       <select name="position" id="position" class="form-control">
          <option value="">위치 선택</option>
          <option value="1">1</option>
          <option value="2">2</option>
          <option value="3">3</option>
          <option value="4">4</option>
          <option value="5">5</option>
          <option value="6">6</option>
          <option value="7">7</option>
          <option value="8">8</option>
       </select>
       <input type = "hidden" name = "test2" value = {{file_name}} /> 
      <Br><br><br>

            <input type="submit" value="Submit" class="btn btn-info">
        </div>
    </form>

</div>
</div>

    </body>
</html>
'''

file = open("templates/showSeg.html","w")
file.write(text)
file.close()

In [47]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
<script src="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.js"></script>
<link href="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.css" rel="stylesheet">
</head>


    <body>
    <div class="container">
        <h2 align="center"> Our Result</h2>
        <br>

<br>


  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 100%;text-align:center">
{% if True %}
	<img src="{{data1}}" alt="our_result_image"  width="700">
{% endif %}
</div>

</div>



</div>

    </body>
</html>
'''

file = open("templates/showResult.html","w")
file.write(text)
file.close()

In [48]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
<script src="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.js"></script>
<link href="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.css" rel="stylesheet">
</head>


    <body>
    <div class="container">
        <h2 align="center"> Pose Transfer - Select the pose</h2>
        <br><br>

<div style="float: left; padding: 10px; width: 100%;text-align:center">
Human Image
</div>
  </div>

  <div class="container; text-align:center">

<div style="float: left; padding: 10px; width: 100%;text-align:center">
{% if True %}
	<img src={{data2}} alt="human_image" width="256">
{% endif %}
</div>
</div>





</div>
<br><br><br>


<div class="container" style="text-align:center">
<div class="row">

    <form action = "/Select_one_pose" method = "POST">
      <input type = "hidden" name = "imgpath" value = {{data2}} /> 
      <input type = "hidden" name = "imgname" value = {{data1}} /> 
      <input type = "hidden" name = "backname" value = {{data0}} /> 

     <input type="radio" name =  "posenum"value="1"> 포즈1  
     <input type="radio" name = "posenum" value="2">  포즈2

      </div>
      <div style = "text-align:center">
            <input type="submit" name='msg' value= "Pose Transfer" class="btn btn-info">
        </div>
        
    </form>

</div>
</div>

    </body>
</html>
'''

file = open("templates/selectPose.html","w")
file.write(text)
file.close()

# flask 코드

In [98]:
%cd /content

/content


In [99]:
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
import shutil
app = Flask(__name__)
run_with_ngrok(app)   


# 파일 업로드 처리
@app.route('/fileUpload', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        files = request.files
        file_names = []
        print(type(files))
        print('-----------')
        for f in files.to_dict(flat=False)['filename[]']:
            f.save('static/inputs/' + secure_filename(f.filename))
            print(f.filename)
            print('파일 업로드 성공!(' ,secure_filename(f.filename) ,')')
            file_names.append(secure_filename(f.filename))

        #return style_transfer(file_names)
        return render_template('selectPose.html',  data0 = file_names[0], data1 = file_names[1], data2 = 'static/inputs/' + file_names[1])



# 업로드 HTML 렌더링
@app.route('/')
def render_file():
    return render_template('upload.html')
    

@app.route('/Select_one_pose', methods = ['GET', 'POST'])
def select_one_pose():
  #print("in select func")
  select = request.form.get('posenum') ################### 선택한 번호 불러옴
  imgpath = request.form.get('imgpath') ################## human image path 불러옴
  imgname = request.form.get('imgname') ################## 원본 human file 이름
  backname = request.form.get('backname') ################ 원본 배경 이름
  imgnames = [backname, imgname]
  #print(imgnames)
  #return backname
  return pose_transfer_func(select, imgpath, imgnames)


def pose_transfer_func(select, imgpath, imgnames):
  resizing_and_pose_map(imgpath,imgnames[1])
  make_pair(select, imgnames[1])
  print("doing pose_tranfer in function")
  test_pose()

  print("output file is in (content/Pose-Transfer/results/fashion_PATN/test_latest/images/) path ")
  ########## outputfile 이름  input human file이랑 똑같게 해주랑 ---> 이게 imgnames[1] 이야
  src = os.path.join('/content/Pose-Transfer/results/fashion_PATN/test_latest/images/', 'output.jpg')
  img = Image.open(src)
  imgcrop = img.crop((704, 0, 176, 256))
  
  os.rename(src, imagenames[1])



  from_pose = '/content/Pose-Transfer/results/fashion_PATN/test_latest/images/' +  imgnames[1]
  to_st = 'static/inputs'
  shutil.copy(from_pose, to_st) # 그러면 output 경로 안에 있었던 파일이 static/inputs로 이동하면서 파일이 덮어씌워질거야 
  
  pad_msg = make_paddding('static/inputs/' + imgnames[1]) ############### 이미지 1 대 1으로 padding 넣기
  print(pad_msg)
  return style_transfer(imgnames)



@app.route('/Select_pos', methods = ['GET', 'POST'])
def select_position():
  human_file = request.form.get('test2')
  select = request.form.get('position')
  splicing_msg = splicing_back(int(select), back_file_path, human_file)
  print(splicing_msg)
  return render_template('showResult.html', data1 = 'static/splicing_outputs/splicing_result.jpg')


@app.route('/Seg_human', methods = ['GET', 'POST'])
def seg_human():
  human_file = request.form.get('test2')

  seg_msg = segmentation(human_file)
  print(seg_msg)
  
  pro_msg, seg_file_path = pre_processing(human_file.split('.')[0] + '_black.png', human_file)
  print(pro_msg)

  return render_template('showSeg.html', data1 = back_file_path, data2 = seg_file_path, file_name = human_file)



def style_transfer(file_names):
  execfile('/content/animegan2-pytorch/anime_execute.py')
  print(file_names)
  global back_file_path
  back_file_path = 'static/anime_outputs/' + file_names[0]
  human_file_path = 'static/anime_outputs/' + file_names[1]
  msg = file_names[1]

  return render_template('showAnime.html', data1 = back_file_path, data2 = human_file_path, file_name = msg)
  #return "style_trnasfer OK"


if __name__ == '__main__':
    # 서버 실행
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://597bdb5303da.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Jun/2021 14:49:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 14:49:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Jun/2021 14:49:04] "GET / HTTP/1.1" 200 -
[2021-06-08 14:49:10,998] ERROR in app: Exception on /fileUpload [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2098, in make_response
    "The view function did not return a valid response. The"
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [08/Jun/20

<class 'werkzeug.datastructures.ImmutableMultiDict'>
-----------
back1.jpg
파일 업로드 성공!( back1.jpg )
human.jpg
파일 업로드 성공!( human.jpg )


127.0.0.1 - - [08/Jun/2021 14:49:14] "GET /static/inputs/human.jpg HTTP/1.1" 200 -


processing 0 / 1 ...
/content/Pose-Transfer/fashion_data/testK /content/static/inputs/human.jpg
doing pose_tranfer in function
/content/Pose-Transfer


usage: ipykernel_launcher.py [-h] [--dataroot DATAROOT]
                             [--batchSize BATCHSIZE] [--loadSize LOADSIZE]
                             [--fineSize FINESIZE] [--input_nc INPUT_NC]
                             [--output_nc OUTPUT_NC] [--ngf NGF] [--ndf NDF]
                             [--which_model_netD WHICH_MODEL_NETD]
                             [--which_model_netG WHICH_MODEL_NETG]
                             [--n_layers_D N_LAYERS_D] [--gpu_ids GPU_IDS]
                             [--name NAME] [--dataset_mode DATASET_MODE]
                             [--model MODEL]
                             [--which_direction WHICH_DIRECTION]
                             [--nThreads NTHREADS]
                             [--checkpoints_dir CHECKPOINTS_DIR] [--norm NORM]
                             [--serial_batches]
                             [--display_winsize DISPLAY_WINSIZE]
                             [--display_id DISPLAY_ID]
                             

# Temp~


In [ ]:
def make_paddding(file_path):
  img = cv2.imread(file_path)
  a, b, c = img.shape

  differ = b - a
  differ_1, differ_2 = 0, 0

  if differ%2 == 0:
    differ_1, differ_2 = abs(differ/2), abs(differ/2)
  else:
    differ_1, differ_2 = abs(differ/2), abs(differ/2) + 1

  change_img = cv2.copyMakeBorder(img, 0, 0, int(differ_1), int(differ_2), cv2.BORDER_CONSTANT)
  cv2.imwrite(file_path, change_img)

  return "make padding OK"